In [1]:
param_date = '20240623'

In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/part.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_.drop_duplicates(['JOIN_PTPKPNCN'], keep='first', ignore_index=True)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'
        for b in range(666):
            try:
                resp = requests.get(input_.loc[a, 'Info'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                if html.xpath('//span[@class="la-btn-v"]/text()')[0].strip() != 'Continue Shopping':
                    continue

                # = = = = = = = = = = = = = = =

                list_tr = html.xpath('//table[@class="moreinfotable"]/tr')

                # = = = = = = = = = = = = = = =

                dict_specification = {}
                for i in range(len(list_tr)):
                    if i == 0:
                        continue

                    list_td = [td.strip() for td in list_tr[i].xpath('./td/text()')]
                    if len(list_td) == 1:
                        dict_specification[str(i-1)] = list_td[0].strip()
                    elif len(list_td) == 2:
                        dict_specification[str(i-1)] = {list_td[0].strip(): list_td[1].strip()}
                    else:
                        break

                json_specification = json.dumps(dict_specification)

                # = = = = = = = = = = = = = = =

                list_tr = html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-1") or (contains(@class, "altrow-b-0")))]/tr')

                # = = = = = = = = = = = = = = =

                dict_kit = {}
                for i in range(len(list_tr)):
                    dict_kit[str(i)] = {}

                    list_td = list_tr[i].xpath('./td')
                    dict_kit[str(i)]['quantity'] = list_td[0].xpath('./text()')[0].strip()
                    dict_kit[str(i)]['component_tag'] = list_td[1].xpath('./text()')[0].strip()

                    json_ = json.loads(list_td[2].xpath('./@value')[0])
                    dict_kit[str(i)]['parttype'] = json_['parttype'].strip()
                    dict_kit[str(i)]['partkey'] = json_['partkey'].strip()

                    json_ = json.loads(list_tr[i].xpath('./input[contains(@id, "listing_data_supplemental[")]/@value')[0])
                    dict_kit[str(i)]['partnumber'] = json_['partnumber'].strip()
                    dict_kit[str(i)]['catalogname'] = json_['catalogname'].strip()

                    dict_kit[str(i)]['url'] = f'''https://www.rockauto.com/en/parts/{dict_kit[str(i)]['partkey']},{dict_kit[str(i)]['partnumber']},{dict_kit[str(i)]['component_tag']},{dict_kit[str(i)]['parttype']}'''

                    a_ = list_tr[i].xpath('./div[@class="listing-text-row-moreinfo-truck"]/a/@href')
                    dict_kit[str(i)]['info'] =  f'''https://www.rockauto.com/en/moreinfo.php?pk={dict_kit[str(i)]['catalogname']}&pt={dict_kit[str(i)]['parttype']}&Lennon=1''' if len(a_) == 0 else a_[0].strip()

                    list_src = [] if len(list_td[3].xpath('./div/input/@value')) == 0 else json.loads(list_td[3].xpath('./div/input/@value')[0])['Slots']
                    dict_src = {}
                    for j in range(len(list_src)):
                        dict_src[str(j)] = 'https://www.rockauto.com' + list_src[j]['ImageData']['Full'].strip()
                    dict_kit[str(i)]['json_src'] = json.dumps(dict_src)

                json_kit = json.dumps(dict_kit)

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'JOIN_PTPKPNCN': input_.loc[a, 'JOIN_PTPKPNCN'],
                                         'Json_Specification': json_specification,
                                         'Json_Kit': json_kit,
                                         'Info': input_.loc[a, 'Info']}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'JOIN_PTPKPNCN': input_.loc[a, 'JOIN_PTPKPNCN'],
                                     'Info': input_.loc[a, 'Info']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'JOIN_PTPKPNCN']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(15):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if len(output_correct) != 0:
    output_correct = output_correct.sort_values(by=['JOIN_PTPKPNCN'], ascending=[True], ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/info-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if len(output_error) != 0:
    output_error = output_error.sort_values(by=['JOIN_PTPKPNCN'], ascending=[True], ignore_index=True)
    output_error.to_excel(f'./{param_date}/info_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：24720

[状态：ok，尝试次数：  1] - 256;14919165;T3;ROCKAUTO
[剩余数量：24705] - [当前时间：15:41:46]

[状态：ok，尝试次数：  1] - 256;14922125;T4;ROCKAUTO
[剩余数量：24704] - [当前时间：15:41:46]

[状态：ok，尝试次数：  1] - 256;15521065;T3C;ROCKAUTO
[剩余数量：24703] - [当前时间：15:41:46]

[状态：ok，尝试次数：  1] - 1002098;15104209;H2;ROCKAUTO
[剩余数量：24702] - [当前时间：15:41:47]

[状态：ok，尝试次数：  1] - 131;15108557;B1;ROCKAUTO
[剩余数量：24701] - [当前时间：15:41:47]

[状态：ok，尝试次数：  1] - 131;10671829;BLACKHAT;ROCKAUTO
[剩余数量：24700] - [当前时间：15:41:47]

[状态：ok，尝试次数：  2] - 131;15108565;B3;ROCKAUTO
[剩余数量：24699] - [当前时间：15:41:48]

[状态：ok，尝试次数：  1] - 260;1285405;GIFTCERTVAR;ROCKAUTO
[剩余数量：24698] - [当前时间：15:41:48]

[状态：ok，尝试次数：  1] - 260;368849;GIFTCERT25;ROCKAUTO
[剩余数量：24697] - [当前时间：15:41:48]

[状态：ok，尝试次数：  1] - 256;15001073;T6;ROCKAUTO
[剩余数量：24696] - [当前时间：15:41:49]

[状态：ok，尝试次数：  3] - 256;15138773;T9;ROCKAUTO
[剩余数量：24695] - [当前时间：15:41:50]

[状态：ok，尝试次数：  2] - 256;15138781;T11;ROCKAUTO
[剩余数量：24694] - [当前时间：15:41:50]

[状态：ok，尝试次数：  3] - 256;15521069;T3R;ROCKAUTO
[剩余数量：